Atver anaconda prompt
conda create -n datiz2 python=3.10
conda activate datiz2


In [4]:
# pip install ipykernel ultralytics torchvision opencv-python matplotlib pandas pycocotools scikit-learn
# Vēl atsevišķi vajadzīgo torch versiju
import os
import json
import shutil
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torch.utils.data import Dataset, DataLoader

# Pārbaudam ierīci
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Izmantotā ierīce: {device}")

Izmantotā ierīce: cpu


In [2]:

import os
import json
import shutil
from pathlib import Path

# KONFIGURĀCIJA
ORIGINAL_PATH = "C:\\Users\\Lietotajs\\Desktop\\gitrepos\\datizrace-proj-2\\dataset-2-seg\\2.3 Auto un ga_je_ju segmenta_cija\\2.3 Auto un gājēju segmentācija"
BASE_DIR = Path("yolo_dataset_v2") # Veidojam jaunu mapi, lai nejauktu ar veco

class_mapping = {
    "car": 0,
    "van": 0,
    "truck": 0,
    "bus": 0,
    "pedestrian": 1,
    "person": 1,
}

def setup_directories():
    if BASE_DIR.exists():
        shutil.rmtree(BASE_DIR)
    
    (BASE_DIR / "images" / "train").mkdir(parents=True, exist_ok=True)
    (BASE_DIR / "images" / "val").mkdir(parents=True, exist_ok=True)
    (BASE_DIR / "labels" / "train").mkdir(parents=True, exist_ok=True)
    (BASE_DIR / "labels" / "val").mkdir(parents=True, exist_ok=True)

def convert_to_yolo(subset_name):
    source_path = Path(ORIGINAL_PATH) / subset_name
    if not source_path.exists():
        print(f"KĻŪDA: Nevar atrast mapi: {source_path}")
        return

    img_dir = source_path / "img"
    ann_dir = source_path / "ann"
    
    json_files = list(ann_dir.glob("*.json"))
    print(f"Apstrādā {len(json_files)} failus priekš '{subset_name}'...")
    
    converted_count = 0
    
    for json_file in json_files:
        img_filename = json_file.stem 
        img_path = img_dir / img_filename
        
        if not img_path.exists():
            continue
            
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            
        img_w = data['size']['width']
        img_h = data['size']['height']
        
        yolo_lines = []
        
        for obj in data['objects']:
            cls_title = obj['classTitle'].lower()
            
            if cls_title not in class_mapping:
                continue 
                
            cls_id = class_mapping[cls_title]
            
            if 'points' in obj and 'exterior' in obj['points']:
                points = obj['points']['exterior']
                line = [str(cls_id)]
                for point in points:
                    x, y = point
                    # Ierobežojam punktus, lai tie neiziet ārpus bildes
                    x = max(0, min(x, img_w))
                    y = max(0, min(y, img_h))
                    
                    line.append(f"{x / img_w:.6f}")
                    line.append(f"{y / img_h:.6f}")
                
                yolo_lines.append(" ".join(line))
            
        if yolo_lines or subset_name == "val":
            shutil.copy(img_path, BASE_DIR / "images" / subset_name / img_filename)
            txt_filename = img_filename.rsplit('.', 1)[0] + ".txt"
            with open(BASE_DIR / "labels" / subset_name / txt_filename, "w") as f:
                f.write("\n".join(yolo_lines))
            converted_count += 1

    print(f"--> Veiksmīgi konvertēti: {converted_count} no {len(json_files)} failiem.")

setup_directories()
convert_to_yolo("train")
convert_to_yolo("val")

Apstrādā 100 failus priekš 'train'...
--> Veiksmīgi konvertēti: 92 no 100 failiem.
Apstrādā 16 failus priekš 'val'...
--> Veiksmīgi konvertēti: 16 no 16 failiem.


In [5]:
import os
dataset_path = os.path.abspath("yolo_dataset_v2")

yaml_content = f"""
path: {dataset_path}
train: images/train
val: images/val
nc: 2
names: ['car', 'pedestrian']
"""

with open("kitti_v2.yaml", "w") as f:
    f.write(yaml_content)

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8m-seg.pt') 

print("Sākam maksimālo apmācību (High Resolution)...")

results = model.train(
    data='kitti_v2.yaml',
    
    epochs=50,
    imgsz=960,
    batch=16,
    
    patience=0,
    optimizer='auto',

    device=0, # GPU
    project='kitti_projekts',
    name='yolo_best_run',
    verbose=True
)

Sākam maksimālo apmācību (High Resolution)...
Ultralytics 8.3.235  Python-3.10.19 torch-2.5.1+cu121 CUDA:0 (NVIDIA L40-16Q, 16384MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=kitti_v2.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo_best_run3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, pati

In [15]:
import os
import cv2
import numpy as np
import torch
import torch.utils.data
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from pathlib import Path

DATA_DIR = Path("yolo_dataset_v2")
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
NUM_CLASSES = 3  # 0=Fons, 1=Auto, 2=Gājējs
EPOCHS = 10

class KittiYoloDataset(torch.utils.data.Dataset):
    def __init__(self, root, subset):
        self.root = root
        self.subset = subset
        self.img_dir = self.root / "images" / subset
        self.lbl_dir = self.root / "labels" / subset
        self.imgs = list(sorted(os.listdir(self.img_dir)))

    def __getitem__(self, idx):
        img_name = self.imgs[idx]
        img_path = self.img_dir / img_name
        img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        height, width, _ = img.shape
        
        lbl_path = self.lbl_dir / (img_name.rsplit('.', 1)[0] + ".txt")
        
        boxes = []
        masks = []
        labels = []
        
        if lbl_path.exists() and lbl_path.stat().st_size > 0:
            with open(lbl_path, 'r') as f:
                lines = f.readlines()
                
            for line in lines:
                parts = list(map(float, line.strip().split()))
                cls_id = int(parts[0])
                coords = parts[1:]
                
                obj_label = cls_id + 1 
                
                poly = []
                for i in range(0, len(coords), 2):
                    px = coords[i] * width
                    py = coords[i+1] * height
                    poly.append([px, py])
                
                poly_np = np.array(poly, dtype=np.int32)
                
                mask = np.zeros((height, width), dtype=np.uint8)
                cv2.fillPoly(mask, [poly_np], 1)
                
                pos = np.where(mask)
                xmin = np.min(pos[1])
                xmax = np.max(pos[1])
                ymin = np.min(pos[0])
                ymax = np.max(pos[0])
                
                if xmax > xmin and ymax > ymin:
                    boxes.append([xmin, ymin, xmax, ymax])
                    masks.append(mask)
                    labels.append(obj_label)

        target = {}
        if len(boxes) > 0:
            target["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)
            target["labels"] = torch.as_tensor(labels, dtype=torch.int64)
            target["masks"] = torch.as_tensor(np.array(masks), dtype=torch.uint8)
        else:
            target["boxes"] = torch.zeros((0, 4), dtype=torch.float32)
            target["labels"] = torch.zeros((0), dtype=torch.int64)
            target["masks"] = torch.zeros((0, height, width), dtype=torch.uint8)
            
        target["image_id"] = torch.tensor([idx])
        
        img_tensor = torchvision.transforms.functional.to_tensor(img)
        
        return img_tensor, target

    def __len__(self):
        return len(self.imgs)

def get_model(num_classes):
    # Ielādēt ResNet50 modeli ar COCO svariem
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")
    
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)
    
    return model

def collate_fn(batch):
    return tuple(zip(*batch))

dataset_train = KittiYoloDataset(DATA_DIR, 'train')
data_loader = torch.utils.data.DataLoader(
    dataset_train, batch_size=16, shuffle=True, collate_fn=collate_fn
)

model_rcnn = get_model(NUM_CLASSES)
model_rcnn.to(DEVICE)

params = [p for p in model_rcnn.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

print(f"Sākam Mask R-CNN apmācību uz {DEVICE} ({EPOCHS} epochas)...")

for epoch in range(EPOCHS):
    model_rcnn.train()
    epoch_loss = 0
    i = 0
    for images, targets in data_loader:
        images = list(image.to(DEVICE) for image in images)
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]
        
        loss_dict = model_rcnn(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
        epoch_loss += losses.item()
        i += 1
        
    lr_scheduler.step()
    print(f"Epoch {epoch+1}/{EPOCHS} | Vidējais Loss: {epoch_loss/i:.4f}")

torch.save(model_rcnn.state_dict(), "mask_rcnn_kitti.pth")
print("Mask R-CNN apmācība pabeigta un modelis saglabāts!")




Sākam Mask R-CNN apmācību uz cuda (10 epochas)...
Epoch 1/10 | Vidējais Loss: 2.5231
Epoch 2/10 | Vidējais Loss: 1.3127
Epoch 3/10 | Vidējais Loss: 1.0859
Epoch 4/10 | Vidējais Loss: 0.9504
Epoch 5/10 | Vidējais Loss: 0.9135
Epoch 6/10 | Vidējais Loss: 0.8876
Epoch 7/10 | Vidējais Loss: 0.8802
Epoch 8/10 | Vidējais Loss: 0.8895
Epoch 9/10 | Vidējais Loss: 0.8885
Epoch 10/10 | Vidējais Loss: 0.8853
Mask R-CNN apmācība pabeigta un modelis saglabāts!


In [7]:
import os
import cv2
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from pathlib import Path
from ultralytics import YOLO
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

DATA_DIR = Path("yolo_dataset_v2")
VAL_IMG_DIR = DATA_DIR / "images" / "val"
OUTPUT_DIR = Path("modelu_salidzinajums")
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
CONF_THRESHOLD = 0.5

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

yolo_weights = Path("kitti_projekts/yolo_best_run3/weights/best.pt")
if not yolo_weights.exists():
    print(f"KĻŪDA: Nevar atrast YOLO svarus: {yolo_weights}")
else:
    print(f"Ielādē YOLO modeli no: {yolo_weights}")
    yolo_model = YOLO(yolo_weights)

def get_model_instance_segmentation(num_classes):
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights=None)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)
    return model

rcnn_weights = Path("mask_rcnn_kitti.pth")
rcnn_model = get_model_instance_segmentation(num_classes=3)

if not rcnn_weights.exists():
    print(f"KĻŪDA: Nevar atrast Mask R-CNN svarus: {rcnn_weights}")
else:
    print(f"Ielādē Mask R-CNN modeli no: {rcnn_weights}")
    rcnn_model.load_state_dict(torch.load(rcnn_weights, map_location=DEVICE))
    rcnn_model.to(DEVICE)
    rcnn_model.eval()

image_files = list(VAL_IMG_DIR.glob("*.png"))
print(f"Sākam apstrādāt {len(image_files)} bildes...")

for i, img_path in enumerate(image_files):
    filename = img_path.name

    img_cv2 = cv2.imread(str(img_path))
    img_rgb = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB)
    
    yolo_results = yolo_model(str(img_path), verbose=False)[0]
    img_yolo_plot = yolo_results.plot()
    img_yolo_plot = cv2.cvtColor(img_yolo_plot, cv2.COLOR_BGR2RGB)

    img_tensor = torchvision.transforms.functional.to_tensor(img_rgb).to(DEVICE).unsqueeze(0)
    with torch.no_grad():
        prediction = rcnn_model(img_tensor)[0]

    img_rcnn_plot = img_rgb.copy()
    masks = prediction['masks'].cpu().numpy()
    scores = prediction['scores'].cpu().numpy()
    labels = prediction['labels'].cpu().numpy()
    boxes = prediction['boxes'].cpu().numpy()

    mask_overlay = np.zeros_like(img_rcnn_plot)
    
    for idx, score in enumerate(scores):
        if score > CONF_THRESHOLD:

            color = (0, 255, 0) if labels[idx] == 1 else (255, 0, 0)

            box = boxes[idx].astype(int)
            cv2.rectangle(img_rcnn_plot, (box[0], box[1]), (box[2], box[3]), color, 2)
            
            mask = masks[idx, 0] > 0.5
            mask_overlay[mask] = color

            label_text = f"{'Car' if labels[idx] == 1 else 'Pedestrian'}: {score:.2f}"
            cv2.putText(img_rcnn_plot, label_text, (box[0], box[1]-5), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    img_rcnn_plot = cv2.addWeighted(img_rcnn_plot, 1, mask_overlay, 0.4, 0)

    fig, axes = plt.subplots(1, 2, figsize=(20, 8))
    
    axes[0].imshow(img_yolo_plot)
    axes[0].set_title("YOLOv8", fontsize=16)
    axes[0].axis('off')
    
    axes[1].imshow(img_rcnn_plot)
    axes[1].set_title("Mask R-CNN", fontsize=16)
    axes[1].axis('off')
    
    save_path = OUTPUT_DIR / f"salidzinajums_{filename}"
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close(fig)

print(f"Visas bildes saglabātas mapē: {OUTPUT_DIR.absolute()}")

Ielādē YOLO modeli no: kitti_projekts\yolo_best_run3\weights\best.pt
Ielādē Mask R-CNN modeli no: mask_rcnn_kitti.pth
Sākam apstrādāt 16 bildes...
Visas bildes saglabātas mapē: c:\Users\Lietotajs\Desktop\gitrepos\datizrace-proj-2\dataset-2-seg\modelu_salidzinajums


In [9]:
import os
import cv2
import numpy as np
import torch
import torch.utils.data
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from pathlib import Path
import random

def collate_fn(batch):
    return tuple(zip(*batch))

class KittiStandardDataset(torch.utils.data.Dataset):

    def __init__(self, root, subset):
        self.root = root
        self.subset = subset
        self.img_dir = self.root / "images" / subset
        self.lbl_dir = self.root / "labels" / subset
        self.imgs = list(sorted(os.listdir(self.img_dir)))

    def __getitem__(self, idx):
        img_name = self.imgs[idx]
        img_path = self.img_dir / img_name
        lbl_path = self.lbl_dir / (img_name.rsplit('.', 1)[0] + ".txt")
        
        img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        height, width, _ = img.shape
        
        boxes = []
        masks = []
        labels = []
        
        if lbl_path.exists() and lbl_path.stat().st_size > 0:
            with open(lbl_path, 'r') as f:
                lines = f.readlines()
            
            for line in lines:
                parts = list(map(float, line.strip().split()))
                obj_label = int(parts[0]) + 1
                coords = parts[1:]
                
                poly = []
                for i in range(0, len(coords), 2):
                    poly.append([coords[i] * width, coords[i+1] * height])
                poly_np = np.array(poly, dtype=np.int32)
                
                mask = np.zeros((height, width), dtype=np.uint8)
                cv2.fillPoly(mask, [poly_np], 1)
                
                pos = np.where(mask)
                if len(pos[1]) > 0 and len(pos[0]) > 0:
                    xmin, xmax = np.min(pos[1]), np.max(pos[1])
                    ymin, ymax = np.min(pos[0]), np.max(pos[0])
                    
                    if xmax > xmin and ymax > ymin:
                        boxes.append([xmin, ymin, xmax, ymax])
                        masks.append(mask)
                        labels.append(obj_label)

        target = {}
        target["image_id"] = torch.tensor([idx])
        
        if len(boxes) > 0:
            target["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)
            target["labels"] = torch.as_tensor(labels, dtype=torch.int64)
            target["masks"] = torch.as_tensor(np.array(masks), dtype=torch.uint8)
        else:
            # Tukšs piemērs
            target["boxes"] = torch.zeros((0, 4), dtype=torch.float32)
            target["labels"] = torch.zeros((0), dtype=torch.int64)
            target["masks"] = torch.zeros((0, height, width), dtype=torch.uint8)
            
        img_tensor = torchvision.transforms.functional.to_tensor(img)
        
        return img_tensor, target

    def __len__(self):
        return len(self.imgs)

def get_model_instance_segmentation(num_classes):
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)
    return model

    
DEVICE = torch.device('cpu') 
DATA_DIR = Path("yolo_dataset_v2") 

model_rcnn = get_model_instance_segmentation(num_classes=3) 
model_rcnn.load_state_dict(torch.load("mask_rcnn_kitti.pth", map_location=DEVICE))
model_rcnn.to(DEVICE)
model_rcnn.eval()

metric_box  = MeanAveragePrecision(iou_type="bbox", class_metrics=True)
metric_mask = MeanAveragePrecision(iou_type="segm", class_metrics=True)

dataset_val = KittiStandardDataset(DATA_DIR, 'val')
data_loader_val = torch.utils.data.DataLoader(
    dataset_val, batch_size=1, shuffle=False, num_workers=0, collate_fn=collate_fn
)

print("Metrikas")

with torch.no_grad():
    for images, targets in data_loader_val:
        images = list(image.to(DEVICE) for image in images)
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

        preds = model_rcnn(images)

        formatted_preds = []
        for p in preds:
            formatted_preds.append({
                "boxes": p["boxes"],
                "scores": p["scores"],
                "labels": p["labels"],
                "masks": p["masks"].squeeze(1) > 0.5
            })

        formatted_targets = []
        for t in targets:
            formatted_targets.append({
                "boxes": t["boxes"],
                "labels": t["labels"],
                "masks": t["masks"]
            })

        metric_box.update(formatted_preds, formatted_targets)
        metric_mask.update(formatted_preds, formatted_targets)

results_box = metric_box.compute()
results_mask = metric_mask.compute()

def print_metrics(results, title_prefix):
    map_50_95_all = results['map'].item()
    map_50_all    = results['map_50'].item()
    per_class     = results['map_per_class']
    
    print(f"\n {title_prefix} METRIKAS")
    print(f"  VISI (mAP 50-95):        {map_50_95_all:.4f}")
    print(f"  VISI (mAP 50):           {map_50_all:.4f}")
    
    if len(per_class) >= 1:
        print(f"  AUTO (mAP 50-95):        {per_class[0].item():.4f}")
    else:
        print(f"  AUTO (mAP 50-95):        Nav datu")
        
    if len(per_class) >= 2:
        print(f"  GĀJĒJI (mAP 50-95):      {per_class[1].item():.4f}")
    else:
        print(f"  GĀJĒJI (mAP 50-95):      Nav datu")

print_metrics(results_box, "BOX (Detekcija)")
print_metrics(results_mask, "MASK (Segmentācija)")

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to C:\Users\Lietotajs/.cache\torch\hub\checkpoints\maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth


100.0%


Metrikas

 BOX (Detekcija) METRIKAS
  VISI (mAP 50-95):        0.2598
  VISI (mAP 50):           0.5493
  AUTO (mAP 50-95):        0.2394
  GĀJĒJI (mAP 50-95):      0.2801

 MASK (Segmentācija) METRIKAS
  VISI (mAP 50-95):        0.1450
  VISI (mAP 50):           0.2827
  AUTO (mAP 50-95):        0.2370
  GĀJĒJI (mAP 50-95):      0.0530
